# Predicting Housing Prices in King County, Washington

#### Griffin Hundley, Malcolm Katzenbach, and Lauren Phipps

***

### Overview  <a id="Overview"></a>

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

[1. Business Problem](#Business-Problem)

[2. Data Understanding](#Data-Understanding)

[3. Data Cleaning](#Data-Cleaning)

[4. Data Modeling](#Data-Modeling)

[5. Evaluation](#Evaluation)

[6. Conclusion](#Conclusion)

***

## Business Problem  <a id="Business-Problem"></a>

When listing a house for sale, it is important to price it accurately. Houses that are priced too high can sit on the market for months, which can lead to the price eventually being dropped to generate sales. According to a study from Zillow, in the Seattle area, a house that is on the market for 2 months will lead to a 5% drop in price. On the other side, if a house is priced too low, a seller is not maximizing the return on their house. In both of these cases, the seller is not getting as much money as they can and the realtor is not maximizing their comission for the sale, as commission is tied to selling price. 

To combat this, we are creating a model to estimate the price of a home based on certain characterisitics of the house. This will allow realtors to more accurately price the house to ensure it sells quickly and at a fair price. 

The questions we are targeting are:

   1) Which house features most impact the price of house?

   2) How do these feature impact the price of a house?

***
Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

Questions to consider:

What are the business's pain points related to this project?
How did you pick the data analysis question(s) that you did?
Why are these questions important from a business perspective?

***

## Data Understanding  <a id="Data-Understanding"></a>

The data being used is publicly available housing data from King County, Washington from 2014-2015. The dataset contains ~21,600 rows, with each row representing the sale of a house in King County. The columns represent features of that house. The target variable is the price of the house, which will be used to determine the effect of several features on houses.  Prior to any data analysis, the test data is split from the training data to prevent data leakage. The split is 80% training, 20% testing.
***

```Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'grade', 'sqft_above', 'yr_built',
       'sqft_living15', 'sqft_lot15', 'has_been_renovated', 'ratio_15',
       'years_old', 'condition_2', 'condition_3', 'condition_4',
       'condition_5'],
      dtype='object')```

Questions to consider:

Where did the data come from, and how do they relate to the data analysis questions?
What do the data represent? Who is in the sample and what variables are included?
What is the target variable?
What are the properties of the variables you intend to use?

Import packages, run code, explore data

In [1]:
# import sys
# import os
# module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from src import datacleaning as dc
# from src import linregvis as lrv
import datacleaning as dc
import linregvis as lrv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split 
from sklearn.dummy import DummyRegressor
from statsmodels.graphics.regressionplots import abline_plot
from sklearn.metrics import mean_squared_error
%matplotlib inline
sns.set_style('darkgrid')
pd.options.display.float_format = "{:,.2f}".format
import warnings
warnings.filterwarnings('ignore')
import matplotlib.ticker as mtick

ModuleNotFoundError: No module named 'datacleaning'

In [ ]:
# Getting the initial data and splitting into test and training sets in an 80:20 split
df = dc.test_split(
    dc.get_data('data/kc_house_data.csv')
)
X_train = df[0]
X_test = df[1]
y_train = df[2]
y_test = df[3]

In [ ]:
X_train.head()

In [ ]:
X_train.describe()

In [ ]:
X_train.columns

## Data Preparation  <a id="Data-Preparation"></a>

The first step was to split the data into the training set and the test set. This was done as an 80%/20% split, respectively.

From here we dropped the following columns: view, lat, long, zipcode, and sqft_basement. We changed the date to a datetime datatype and filled in the null values for waterfront as zeroes since most homes are not waterfront and it would likely be something that was specified if so. We also created two new features. First is ratio15, which is the ratio of the size of the house compared to the homes around it, using the sqft_living and sqft_living15. The other is years old, with the year built subtracted from 2020. Lastly, we created dummy variables for the conditions. 

***
Questions to consider:

Were there variables you dropped or created?
How did you address missing values or outliers?
Why are these choices appropriate given the data and the business problem?


In [ ]:
# Data cleaning, dropping unused columns, feature engineering, dummy columns
X_train = dc.clean_it(df[0])
X_test = dc.clean_it(df[1])

## Data Modeling  <a id="Data-Modeling"></a>

The first step in modeling was to look at the relationship between price (the target variable) and each of the features in the data set. This was done by creating a pairplot. These pairplots show that there is a somewhat linear relationship between price and bathrooms, living space (sqft_living), and grade. Describe and justify the process for analyzing or modeling the data.
***
Questions to consider:

How did you analyze or model the data?
How did you iterate on your initial approach to make it better?
Why are these choices appropriate given the data and the business problem?

Run code to model data

***

### Assessment of Linearity Between Price and Features

In [ ]:
# Break the columns in groups to plot 4 on a row at a time
df = pd.concat([X_train, y_train], axis = 1)
n = 4
features = list(X_train.drop(columns = 'date').columns)
row_groups= [features[i:i+n] for i in range(0, len(features), n) ]
for i in row_groups:
    pp = sns.pairplot(data=df, y_vars=['price'],x_vars=i, kind="reg", height=3)

The relationship between living space and price looks the strongest so that was the feature we started to investigate further. Looking at just that variable we made a regression plot that also shows the distribution of each variable. 

In [ ]:
df = pd.concat([X_train, y_train], axis = 1)
fig = sns.jointplot(x="sqft_living", y="price", data=df, kind="reg", height = 8, ratio = 5)
fig.fig.suptitle("Relationship Between Living Space (ft2) and House Price", fontsize = 20)
fig.fig.subplots_adjust(top=0.95)
fig.set_axis_labels(xlabel = 'Living Space, (ft2)', ylabel = 'Price', fontsize = 14)
sns.set(font_scale = 1)

There is a linear relationship between the variables and both distributions have a slight skew. 


***
### Assessment of Correlation Between Features

Finally, we created a heat map to investigate the linearity within the features to address any possible issues of colinearity in our model as we add in additional features. Any features with a correlation value of greater than 0.7 will not be used together in iterations of a model. 


In [ ]:
fig,ax = plt.subplots(figsize = (10, 10))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(df.corr(), cmap = cmap)
ax.set_title('Heat Map of Correlation Between Predictors', fontsize = 20)
fig.savefig('./images/heatmap.png', bbox_inches='tight')

***

### Model Iteration

#### Creation of Dummy Model

First we created a dummy regressor model as a baseline. The dummy regressor uses the default of predicting the mean of the training set and was fitted to the our training X and y. We also checked the baseline score of the model.

In [ ]:
dummy = DummyRegressor()
dummy.fit(X_train, y_train)
dummy.score(X_train, y_train)

To find our baseline error, we predicted the y values with our model for the X_train values. My using the mean_squared_error from sklearn metrics, we found by Root Mean Square Error between our predicted and known y_train values.

In [ ]:
y_pred = dummy.predict(X_train)
dummy_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
dummy_rmse

#### Model 1

The first model we checked was by setting only variable to sq_ft_living. This variable was chosen due to how most people commonly think of the square footage of the house. By using the Ordinary Least Squares function we fitted, we created our model.

In [ ]:
formula = "price ~ sqft_living"
model_1 = lrv.run_lr(formula, X_train, y_train)

***

From the summary of our model we first check the R-squared value, which tells us the percentage of the variance in price can be explained by this model. It can only explain 48.9% of the variance in price. Then by checking the p value for the coefficient of the sqft_living variable to check if it is signficant to our model or not. Using a confidence level of 0.05, because the p value is less than 0025, we can reject the null hypothesis that the coefficient is not significantly different from zero. The coefficient for sqft_living tells us that for every square foot in the house, the price of the house increase by $281.61. 

Next we create the y prediction by using the X_train values using this first model. The Root Mean Squared Error is then calculated.

In [ ]:
y_pred = model_1.predict(X_train)
model_1_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
model_1_rmse

The Root Mean Squared Error (RMSE) is equal to 265871.88, which is less than the RMSE for the baseline model. This indicates that this model is better than our baseline model.

With the first model being only a basic linear regression model, we graph a scatter plot of the price vs. sqft_living and draw the regression line.

In [ ]:
fig, ax = plt.subplots(figsize = (14,8))
ax = sns.scatterplot(X_train.sqft_living, y_train)
ax.set_xlabel('Square Foot Living Space (ft2)', fontsize = 14)
ax.set_ylabel('Price', fontsize = 14)
ax.set_title('Effect of living space on Housing Price', fontsize = 24)
ax.tick_params(labelsize = 14)
lrv.dollar_ticks(ax)
abline_plot(model_results=model_1, ax = ax);
fig.savefig('./images/simple_linear_regression.png', bbox_inches='tight')

#### Model 2

The second model expands on the previous model by checking the condition of the house. Using the condition variable we created dummy variables for each level of condition of which there are 5. Using the ols function and fitting the data, we created the second model with price the dependent variable and sqft_living and our condition dummy variables as the independent variables.

In [ ]:
formula = "price ~ sqft_living + condition_2 + condition_3 + condition_4 + condition_5"
model_2 = lrv.run_lr(formula, X_train, y_train)

In model 2, the R-squared value is 0.496. This is only .008 better than the previous model. When the p values are examined, the sqft_living is still equal to 0.000, however the conditions all have a p value greater than 0.1. Keeping are confidence levels the same, we cannot reject the null hypothesis that the coefficient is significantly different from zero. So we can conclude that condition feature is not significantly important to the price value.

In [ ]:
y_pred = model_2.predict(X_train)
model_2_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
model_2_rmse

Checking the RMSE of model two confirms what are R-squared value implied. The RMSE decreased 1000 from the previous model, which is not significant.

#### Model 3

In the third model, the condition dummy variables are removed from the formula due to not being significant and expanded by adding grade, waterfront, years_old, and ratio_15. 

In [ ]:
formula = "price ~ sqft_living + grade + waterfront + years_old + ratio_15"
model_3 = lrv.run_lr(formula, X_train, y_train)

In model 3, the R-squared value is 0.636. This is an increase of 0.14 or 14% than the previous model. The p values for all the features are 0.000, thus for all coefficients the null hypothesis that the coefficient is not significantly different from zero is rejected. All the new features are significant in the prediction of price and increases the percentage of variance in price that the model is able to explain.

In [ ]:
y_pred = model_3.predict(X_train)
model_3_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
model_3_rmse

The RMSE from model 3 is 224526, which is a decrease of nearly 40000. In addition to the R-squared value, this model is much better at predicting the price of homes.

#### Model 4

In the fourth model, the sqft_living is removed due the high mult-linear correlation between the variable and grade and ratio_15. Bathrooms has a high multi-linear correlation with sqft_living, but not as high as with grade or ratio_15. We use bathrooms as a possible replacement for sqft_living.

In [ ]:
formula = "price ~ bathrooms + grade + waterfront + years_old + ratio_15"
model_4 = lrv.run_lr(formula, X_train, y_train)

The fourth model has an R-squared value of 0.594. This is less than the 0.636 of the previous model. Though it decreases possible error due to multi-linear correlation. The p values for the features used are all 0.000, thus rejecting the null hypothesis that the coefficient is not significantly different from zero. Taking into account the multi-linear correlation, this model is the best improvement from the base model.

In [ ]:
y_pred = model_4.predict(X_train)
model_4_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
model_4_rmse

The RMSE is equal 236951 which is an increase from model three, but still a 30000 decrease from model 2. 

#### Model 5

For the fifth model, the feature has_been_renovated was added to see if renovating a house has a predictable impact on the price.

In [ ]:
formula = "price ~ bathrooms + grade + waterfront + years_old + ratio_15 + has_been_renovated"
model_5 = lrv.run_lr(formula, X_train, y_train)

The fifth model gives an identical $R^2$ value, and the p-value is greater than the significance level $\alpha=0.05$ so this is not a statistically significant relationship

In [ ]:
y_pred = model_5.predict(X_train)
model_5_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
model_5_rmse

The RMSE is barely a decrease at all from the previous model.

#### Model 6

For the sixth model, because has_been_renovated and ratio_15 had little impact on the effect, they are removed.

In [ ]:
formula = "price ~ bathrooms + grade + waterfront + years_old"
model_6 = lrv.run_lr(formula, X_train, y_train)

In [ ]:
y_pred = model_6.predict(X_train)
model_6_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
model_6_rmse

In [ ]:
final_model = model_6
test = X_test.sample(1, random_state = 500)
coef = final_model.params
est_price = coef[0] + coef[1]* test.bathrooms + coef[2]* test.grade + coef[3]* test.waterfront + coef[4]*test.years_old
print(f'The estimate price is ${int(est_price)}')
print(f'The real price is ${y_test[test.index[0]]}')

In [ ]:
final_model.params

***

### Validation of Model

#### Analysis of Residuals

In [ ]:
mmean = final_model.resid.mean()
mmin = final_model.resid.min()
mmax = final_model.resid.max()
normalized_residuals = (final_model.resid - mmean)/(mmax - mmin)

t = final_model.fittedvalues
tmean = final_model.resid.mean()
tmin = final_model.resid.min()
tmax = final_model.resid.max()
normalized_predicted = (t - tmean)/(tmax - tmin)

#### The assumption of normality

In [ ]:
fig, ax = plt.subplots(figsize = (14,8))
ax.set_xlabel('Normalized Residuals', fontsize = 14)
ax.set_ylabel('Frequency', fontsize = 14)
ax.set_title('Frequency of Normalized Residuals', fontsize = 24)
ax.set(xlim=(-0.25,.25))
sns.distplot(normalized_residuals, bins = 100);
#fig.savefig('./images/residuals_hist.png', bbox_inches='tight')

In the figure above the residuals of the model are roughly normally distributed, so it passes the assumption of normality.

#### Homoskedasticity Test

In [ ]:
fig, ax = plt.subplots(figsize = (14,8))
ax.set_xlabel('Normalized Predicted Price', fontsize = 14)
ax.set_ylabel('Normalized Residuals', fontsize = 14)
ax.set_title('Residuals vs Predicted Values for Homoskedasticity', fontsize = 24)
sns.scatterplot(x = normalized_predicted, y = normalized_residuals);
#fig.savefig('./images/homoskedasticity.png', bbox_inches='tight')

From the homoskedasticity test, there is an apparent bias.  This implies that there are underlying variables that influence the price that are not captured in the features of this particular model. 

#### QQ Plot

In [ ]:
fig, ax = plt.subplots(figsize = (12, 8))
stats.probplot(normalized_residuals, plot=plt)
ax.set_title('QQ Plot Probability of residuals', fontsize = 24)
ax.set_xlabel('Residuals', fontsize = 14)
ax.set_ylabel('Probability', fontsize = 14)
ax.legend(['Residuals','Normal line']);
#fig.savefig('./images/qqplot.png', bbox_inches='tight')

The probability plot shows the probability of the residuals of being normally distributed.  The tails of the residuals tend away from the normality line, implying that the residuals become less normal.

## Evaluation  <a id="Evaluation"></a>

In [ ]:
y_pred = final_model.predict(X_test)
final_model_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
final_model_rmse

For this particular set of training data, the model was able to predict the test data slightly better than the training data, which implies that this model is generalizable to new incoming data.

Evaluate how well your work solves the stated business problem.

Questions to consider:

How do you interpret the results?
How well does your model fit your data? How much better is this than your baseline model?
How confident are you that your results would generalize beyond the data you have?
How confident are you that this model would benefit the business if put into use?

***

## Conclusion  <a id="Conclusion"></a>

Provide your conclusions about the work you've done, including any limitations or next steps.

Questions to consider:

What would you recommend the business do as a result of this work?
What are some reasons why your analysis might not fully solve the business problem?
What else could you do in the future to improve this project?

***
